<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100"/>

# Cours TAL - Laboratoire 6
# Trois méthodes de désambiguïsation lexicale

**Objectif**

L'objectif de ce laboratoire est d'implémenter et de comparer plusieurs méthodes de désambiguïsation lexicale (en anglais, *Word Sense Disambiguation* ou WSD).  Vous utiliserez un corpus avec plusieurs milliers de phrases, chaque phrase contenant une occurrence du mot anglais *interest* annotée avec le sens que ce mot possède dans la phrase respective.  Les trois méthodes sont les suivantes (elles seront détaillées par la suite) :

* Algorithme de Lesk simplifié.
* Utilisation de word2vec.
* Classification supervisée utilisant des traits lexicaux.

Les deux premières méthodes n'utilisent pas l'apprentissage automatique.  Elles fonctionnent selon le même principe : comparer le contexte d'une occurrence de *interest* avec chacune des définitions des sens (*synsets*) et choisir la définition la plus proche du contexte.  L'algorithme de Lesk définit la proximité comme le nombre de mots en commun, alors que word2vec la calcule comme la similarité de vecteurs.  La dernière méthode vise à classifier les occurrences de *interest*, les sens étant les classes, et les attributs étant les mots du contexte (apprentissage supervisé).

## 1. Analyse des données

Téléchargez le corpus *interest* depuis le [site du Prof. Ted Pedersen](http://www.d.umn.edu/~tpederse/data.html) (il se trouve en bas de sa page web).  Téléchargez l'archive ZIP marquée *original format without POS tags* et extrayez le fichier `interest-original.txt`.  Téléchargez également le fichier `README.int.txt` indiqué à la ligne au-dessus. Veuillez répondre brièvement aux questions suivantes :

a. Quelles sont les URL du fichier ZIP et celle du fichier `README.int.txt` ?

b. Quel est le format du fichier `interest-original.txt` et comment sont annotés les sens de *interest* ?

c. Est-ce qu'il y a aussi des occurrences au pluriel (*interests*) à traite ?

d. Comment sont annotées les phrases qui contiennent plusieurs occurrences du mot *interest* ?

> Veuillez répondre ici (en commentaire) à la question.

a. Voici l'URL du ZIP : https://www.d.umn.edu/~tpederse/Data/interest-original.nopos.tar.gz <br>Voici l'URL du fichier : https://www.d.umn.edu/~tpederse/Data/README.int.txt

b. C'est un fichier texte où chaque phrase est délimitée par une ligne contenant "$$", chaque sens est annoté comme ceci interest_{numéro du sens}

c. Oui, le pluriel est le singulier sont traités

d. Une occurrence par phrase est annotée avec un tag de sens (ex: _1 à _6). Les autres occurrences du mot "interest" dans la même phrase sont marquées par un astérisque (*) placé juste avant le mot (ex: *interest) pour indiquer qu'elles ne sont pas l'objet de l'annotation de sens pour cette instance spécifique du corpus. Cela permet de respecter la consigne "une occurrence étiquetée par phrase" tout en conservant le contexte phrastique complet.

**1e.** D'après le fichier `README.int.txt`, quelles sont les définitions des six sens de *interest* annotés dans les données et quelles sont leurs fréquences ? Vous pouvez copier/coller l'extrait de `README`ici.

> Veuillez répondre ici (en commentaire) à la question.
```text
Sense 1 =  361 occurrences (15%) - readiness to give attention
Sense 2 =   11 occurrences (01%) - quality of causing attention to be given to
Sense 3 =   66 occurrences (03%) - activity, etc. that one gives attention to
Sense 4 =  178 occurrences (08%) - advantage, advancement or favor
Sense 5 =  500 occurrences (21%) - a share in a company or business
Sense 6 = 1252 occurrences (53%) - money paid for the use of money
	  ----
	  2368 occurrences in the sense tagged corpus, where each
	occurrence is a single sentence that contains the word 'interest'.
```

**1f.** De quel dictionnaire viennent les sens précédents ? Où peut-on le consulter en ligne ?  Veuillez aligner les définitions du dictionnaire avec les six sens annotés en écrivant par exemple `Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"`.

> Veuillez répondre ici (en commentaire) à la question.

Le dictionnaire est "The first edition of Longman's Dictionary of Contemporary English", on peut le consulter ici https://www.ldoceonline.com/dictionary/
```text
Sense 1 = "if you have an interest in something or someone, you want to know or learn more about them"
Sense 2 = "a quality or feature of something that attracts your attention or makes you want to know more about it"
Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"
Sense 4 = "the things that bring advantages to someone or something"
Sense 5 = "if you have an interest in a particular company or industry, you own shares in it"
Sense 6 = "the extra money that you must pay back when you borrow money" & "money paid to you by a bank or financial institution when you keep money in an account there"
```

**1g.** En consultant [WordNet en ligne](http://wordnetweb.princeton.edu/perl/webwn), trouvez les définitions des synsets  pour le **nom commun** *interest*.  Combien de synsets y a-t-il ?  Veuillez indiquer comme avant la **définition** de chaque synset pour chacun des six sens ci-dessus (au besoin, fusionner ou ignorer des synsets).

> Veuillez répondre ici (en commentaire) à la question.

Il y a 7 synsets: 

```text
Sense 1 = "a sense of concern with and curiosity about someone or something"
Sense 2 = "the power of attracting or holding one's attention (because it is unusual or exciting etc.)"
Sense 3 = "a diversion that occupies one's time and thoughts (usually pleasantly)"
Sense 4 = "a reason for wanting something done"
Sense 5 = "a right or legal share of something; a financial involvement with something"
Sense 6 = "a fixed charge for borrowing money; usually a percentage of the amount borrowed"
```

On a ignoré le sens suivant : "(usually plural) a social group whose members control some field of activity and who have common aims"

**1h.** Définissez (manuellement, ou avec quelques lignes de code) une liste nommée `senses1` avec les mots des définitions du README, en supprimant les stopwords (p.ex. les mots < 4 lettres).  Affichez la liste.

In [2]:
import nltk

In [3]:
from random import randrange

In [4]:
# Veuillez répondre ici à la question et créer la variable 'senses1' (liste de 6 listes de chaînes).
senses1 = [
    "readiness give attention",
    "quality causing attention given",
    "activity attention",
    "advantage advancement favor",
    "share company business",
    "money paid money"
]
print(senses1)

['readiness give attention', 'quality causing attention given', 'activity attention', 'advantage advancement favor', 'share company business', 'money paid money']


**1i.** En combinant les définitions obtenues aux points (4) et (5) ci-dessus, construisez une liste nommée `senses2` avec pour chacun des sens de *interest* une liste de **mots-clés** correspondants.  Vous pouvez concaténer les définitions, puis écrire des instructions en Python pour extraire les mots (uniques).  Respectez l'ordre des sens données par `README`, et à la fin affichez `senses2`.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

try:
    stop_words_english = stopwords.words('english')
except LookupError:
    nltk.download('stopwords')
    stop_words_english = stopwords.words('english')

try:
    word_tokenize("test")
except LookupError:
    nltk.download('punkt')

LDOCE = [
    "if you have an interest in something or someone, you want to know or learn more about them",
    "a quality or feature of something that attracts your attention or makes you want to know more about it",
    "an activity that you enjoy doing or a subject that you enjoy studying",
    "the things that bring advantages to someone or something",
    "if you have an interest in a particular company or industry, you own shares in it",
    "the extra money that you must pay back when you borrow money" + " " + "money paid to you by a bank or financial institution when you keep money in an account there"
]

WNET = [
    "a sense of concern with and curiosity about someone or something",
    "the power of attracting or holding one's attention (because it is unusual or exciting etc.)",
    "a diversion that occupies one's time and thoughts (usually pleasantly)",
    "a reason for wanting something done",
    "a right or legal share of something; a financial involvement with something",
    "a fixed charge for borrowing money; usually a percentage of the amount borrowed"
]

combined_definitions_text = []
for i in range(len(LDOCE)):
    combined_definitions_text.append(LDOCE[i] + " " + WNET[i])

senses2 = []

stop_words = set(stopwords.words('english'))
stop_words.add('etc')
punctuation_chars = set(string.punctuation)


for definition_text in combined_definitions_text:
    text_lower = definition_text.lower()

    tokens = word_tokenize(text_lower)

    keywords_for_this_sense = []
    for token in tokens:
        if token.isalpha() and token not in stop_words:
            keywords_for_this_sense.append(token)

    unique_keywords = sorted(list(set(keywords_for_this_sense)))
    senses2.append(unique_keywords)

print(senses2)

[['concern', 'curiosity', 'interest', 'know', 'learn', 'sense', 'someone', 'something', 'want'], ['attention', 'attracting', 'attracts', 'exciting', 'feature', 'holding', 'know', 'makes', 'one', 'power', 'quality', 'something', 'unusual', 'want'], ['activity', 'diversion', 'enjoy', 'occupies', 'one', 'pleasantly', 'studying', 'subject', 'thoughts', 'time', 'usually'], ['advantages', 'bring', 'done', 'reason', 'someone', 'something', 'things', 'wanting'], ['company', 'financial', 'industry', 'interest', 'involvement', 'legal', 'particular', 'right', 'share', 'shares', 'something'], ['account', 'amount', 'back', 'bank', 'borrow', 'borrowed', 'borrowing', 'charge', 'extra', 'financial', 'fixed', 'institution', 'keep', 'money', 'must', 'paid', 'pay', 'percentage', 'usually']]


**1j.** Chargez les données depuis `interest-original.txt` dans une liste appelée `sentences` qui contient pour chaque phrase la liste des mots (sans les séparateurs *$$* et *===...*).  Ces phrases sont-elles déjà tokenisées en mots ?  Sinon, faites-le.  À ce stade, ne modifiez pas encore les occurrences annotées *interest(s)\_X*.  Comptez le nombre total de phrases et affichez-en trois au hasard.

In [11]:
# Veuillez répondre ici à la question.
sentences = []

filename = "interest-original.txt"

sentences = []
raw_lines_count = 0
with open(filename, 'r', encoding='utf-8') as f:
    current_sentence_words = []
    for line in f:
        raw_lines_count += 1
        line_stripped = line.strip()

        if line_stripped == "$$":
            if current_sentence_words:
                sentences.append(current_sentence_words)
                current_sentence_words = []
        elif line_stripped.startswith("======================================"):
            continue
        elif line_stripped:
            words_in_line = line_stripped.split()
            current_sentence_words.extend(words_in_line)

    if current_sentence_words:
        sentences.append(current_sentence_words)
        
print("Il y a {} phrases.\nEn voici 3 au hasard :".format(len(sentences)))
print(sentences[151:154])

Il y a 1228 phrases.
En voici 3 au hasard :
[['brunswick', 'also', 'has', 'interests_5', 'in', 'defense', 'and', 'aerospace', 'products', '.'], ['market', 'analysts', 'said', 'that', 'continued', 'declines', 'and', 'volatility', 'in', 'the', 'stock', 'market', 'contributed', 'generally', 'to', 'enhanced', 'investor', 'interest_1', 'in', 'precious', 'metals', ',', 'but', 'that', 'the', 'british', 'political', 'news', 'was', 'the', 'main', 'factor', 'boosting', 'prices', ':', 'chancellor', 'of', 'the', 'exchequer', 'nigel', 'lawson', 'resigned', '.'], ['but', 'he', 'also', 'noted', 'that', 'if', 'uncertainty', 'continues', 'in', 'the', 'stock', 'market', ',', 'that', 'might', 'create', 'some', 'investor', 'interest_1', 'in', 'precious', 'metals', '.']]


## 2. Algorithme de Lesk simplifié

**2a.** Définissez une fonction `wsd_lesk(senses, sentence)` qui prend deux arguments : une liste de listes de mots-clés (comme `senses1` et `senses2` ci-dessus) et une phrase avec une occurrence annotée de *interest* ou *interests*, et qui retourne l'index du sens le plus probable (entre 1 et 6) selon l'algorithme de Lesk.  Cet algorithme choisit le sens qui a le maximum de mots en commun avec le contexte de *interest*.  Vous pouvez choisir vous-mêmes la taille de ce voisinage (`window_size`).  En cas d'égalité entre deux sens, tirer la réponse au sort.

In [61]:
# Veuillez répondre ici à la question.

import nltk
from nltk.corpus import stopwords
import random

try:
    stop_words_english = stopwords.words('english')
except LookupError:
    nltk.download('stopwords')
    stop_words_english = stopwords.words('english')


def wsd_lesk(senses, sentence, window_size=5):
    stop_words = set(stopwords.words('english'))

    target_word_index = -1
    for i, token in enumerate(sentence):
        if token.startswith("interest_") or token.startswith("interests_"):
            target_word_index = i
            break

    if target_word_index == -1:
        return None

    start_index = max(0, target_word_index - window_size)
    end_index = min(len(sentence), target_word_index + window_size + 1)
    context_tokens_raw = sentence[start_index:target_word_index] + sentence[target_word_index+1:end_index]

    context_cleaned = set()
    for token in context_tokens_raw:
        token_lower = token.lower()
        if token_lower.isalpha() and token_lower not in stop_words:
            context_cleaned.add(token_lower)
    
    if not context_cleaned:
        return random.randint(1, len(senses))

    best_sense_indices = []
    max_overlap = -1

    for i, sense_kws in enumerate(senses):
        sense_kws_set = set(sense_kws)
        overlap = len(context_cleaned.intersection(sense_kws_set))

        if overlap > max_overlap:
            max_overlap = overlap
            best_sense_indices = [i + 1]
        elif overlap == max_overlap and max_overlap != -1:
            best_sense_indices.append(i + 1)
    
    if not best_sense_indices:
        return random.randint(1, len(senses))
    elif len(best_sense_indices) == 1:
        return best_sense_indices[0]
    else:
        return random.randint(1, len(senses))


print(wsd_lesk(senses1, sentences[11]))
print(wsd_lesk(senses2, sentences[11]))

print(senses2[4])
print(sentences[11])

2
5
['company', 'financial', 'industry', 'interest', 'involvement', 'legal', 'particular', 'right', 'share', 'shares', 'something']
['last', 'year', ',', 'mitsubishi', 'international', 'corp.', ',', 'the', 'new', 'york-based', 'arm', 'of', 'mitsubishi', 'corp.', ',', 'bought', 'controlling', 'interest_5', 'in', 'the', 'glass', 'company', 'in', 'a', 'joint', 'venture', 'with', 'ronald', 'bodner', ',', 'a', 'glass', 'industry', 'executive', 'and', 'mitsubishi', 'consultant', '.']


**2b.** Définissez maintenant une fonction `evaluate_wsd(fct_name, senses, sentences)` qui prend en paramètre le nom de la méthode de similarité (pour commencer : `wsd_lesk`) ainsi que la liste des mots-clés par sens, et la liste de phrases, et qui retourne le score de la méthode de similarité.  Ce score sera tout simplement le pourcentage de réponses correctes (sens trouvé identique au sens annoté).

In [74]:
# Veuillez répondre ici à la question.
import random
from nltk.corpus import stopwords # Nécessaire si wsd_lesk l'utilise en interne

def evaluate_wsd(wsd_function, senses_keywords, sentences_data):
    correct_predictions = 0
    total_predictions = 0

    for sentence_tokens in sentences_data:
        true_sense = None
        for token in sentence_tokens:
            if token.startswith("interest_") or token.startswith("interests_"):
                try:
                    sense_part = token.split('_')[-1]
                    numeric_sense_str = ""
                    for char in sense_part:
                        if char.isdigit():
                            numeric_sense_str += char
                        else:
                            break
                    if numeric_sense_str:
                        true_sense = int(numeric_sense_str)
                    break
                except (IndexError, ValueError):
                    continue

        if true_sense is None:
            continue

        total_predictions += 1
       
        predicted_sense = wsd_function(senses_keywords, sentence_tokens)
        
        if predicted_sense is not None and predicted_sense == true_sense:
            correct_predictions += 1

    if total_predictions == 0:
        return 0.0

    accuracy = (correct_predictions / total_predictions) * 100
    return accuracy

score = evaluate_wsd(wsd_lesk, senses2, sentences)
print(f"Score de la méthode 'wsd_lesk': {score:.2f}%")

Score de la méthode 'wsd_lesk': 25.08%


**2c.** En fixant au mieux la taille de la fenêtre autour de *interest*, quel est le meilleur score de la méthode de Lesk simplifiée ?  Quelle liste de sens conduit à de meilleurs scores, `senses1` ou `senses2` ?

In [ ]:
# Veuillez répondre ici à la question.


## 3. Utilisation de word2vec pour la similarité contexte vs. synset

**3a.** En réutilisant une partie du code de `wsd_lesk`, veuillez maintenant définir une fonction `wsd_word2vec(senses, sentence)` qui choisit le sens en utilisant la similarité **word2vec** étudiée dans le labo précédent. 
* Vous pouvez chercher dans la [documentation des KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) comment calculer directement la similarité entre deux listes de mots.
* Comme `wsd_lesk`, la nouvelle fonction `wsd_word2vec` prend en argument une liste de listes de mots-clés par sens (comme `senses1` et `senses2` ci-dessus), et une phrase avec une occurrence annotée de *interest* ou *interests*.
* La fonction retourne le numéro du sens le plus probable selon la similarité word2vec entre les mots du sens et ceux du voisinage de *interest*.  En cas d'égalité, tirer le sens au sort.
* Vous pouvez régler la taille du voisinage (`window_size`) par l'expérimentation.  

In [ ]:
import gensim
from gensim.models import KeyedVectors
path_to_model = "../../../gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz" # à adapter
wv_model = gensim.models.KeyedVectors.load_word2vec_format(path_to_model, binary=True)  # C bin format

In [ ]:
# Veuillez répondre ici à la question.


**3b.** Appliquez maintenant la même méthode `evaluate_wsd` avec la fonction `wsd_word2vec` (en cherchant une bonne valeur de la taille de la fenêtre) et affichez le score de la similarité word2vec.  Comment se compare-t-il avec le score précédent (Lesk) ?

In [ ]:
# Veuillez répondre ici à la question.


## 4. Classification supervisée avec des traits lexicaux
Vous entraînerez maintenant des classifieurs pour prédire le sens d'une occurrence dans une phrase.  Le premier but sera de transformer chaque phrase en un ensemble d'attributs pour formater les données en vue des expériences de classification.

Veuillez utiliser le classifieur `NaiveBayesClassifier` fourni par NLTK.  Le mode d'emploi se trouve dans le [Chapitre 6, sections 1.1-1.3](https://www.nltk.org/book/ch06.html) du livre NLTK.  Consultez-le attentivement pour trouver comment formater les données.  De plus, il faudra séparer les données en sous-ensembles d'entraînement et de test.

On vous propose de nommer les attributs `word-k`, ..., `word-2`, `word-1`, `word+1`, `word+2`, ..., `word+k` (fenêtre de taille `2*k` autour de *interest*).  Leurs valeurs sont les mots observés aux emplacements respectifs, ou `NONE` si la position dépasse l'étendue de la phrase.  Vous ajouterez un attribut nommé `word0` qui est l'occurrence du mot *interest* au singulier ou au pluriel.  

Pour chaque occurrence de *interest*, vous devrez donc créer la représentation suivante (où `6` est le numéro du sens, essentiel pour l'entraînement, mais à cacher lors de l'évaluation) :
```
[{'word-1': 'in', 'word+1': 'rates', 'word-2': 'declines', 'word+2': 'NONE', 'word0': 'interest'}, 6]
```

**4a.** En partant de la liste des phrases appelée `sentences` préparée plus haut, veuillez générer la liste avec toutes les représentation, appelée `items_with_features`.  Vous pouvez vous aider du livre NLTK.

In [ ]:
# Veuillez répondre ici à la question.

print(len(items_with_features))
print(items_with_features[151:154])

**4b.** Veuillez séparer les données aléatoirement en 80% pour l'entraînement et 20%  pour l'évaluation.  Veuillez faire une division stratifiée : les deux sous-ensembles doivent contenir les mêmes proportions de sens que l'ensemble de départ.  Ils seront appelés `iwf_train` et `iwf_test`.

In [ ]:
from random import shuffle

In [ ]:
iwf_train = []
iwf_test  = []
# Veuillez répondre ici à la question.

print(len(iwf_train), ' ', len(iwf_test))
print(iwf_test[:2], iwf_test[-2:])

**4c.** Veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_train` et la tester sur `iwf_test` (voir la documentation NLTK).  En expérimentant avec différentes largeurs de fenêtres, quel est le meilleur score que vous obtenez (avec la fonction `accuracy` de NLTK) sur l'ensemble de test ?  Comment se compare-t-il avec les précédents ?

In [ ]:
from nltk.classify import naivebayes 
# Veuillez répondre ici à la question.


**4d.** En utilisant la fonction `show_most_informative_features()`, veuillez afficher les attributs les plus informatifs et commenter le résultat.

**4e.** On souhaite également obtenir les scores pour chaque sens.  Pour ce faire, il faut demander les prédictions une par une au classifieur (voir le [livre NLTK](https://www.nltk.org/book/ch06.html)), et comptabiliser les prédictions correctes pour chaque sens.  Vous pouvez vous inspirer de `evaluate_wsd`, et écrire une fonction `evaluate_wsd_supervised(classifier, items_with_features)`, que vous appliquerez aux donnés `iwf_test`.  Veuillez afficher ces scores.

In [ ]:
def evaluate_wsd_supervised(classifier, items_with_features):
# à compléter

evaluate_wsd_supervised(classifier, iwf_test)

## 5. Conclusion

Veuillez recopier ci-dessous, en guise de conclusion, les scores des trois expériences réalisées, pour pouvoir les comparer d'un coup d'oeil.  Quel est le meilleur score obtenu?

## Fin du laboratoire

Merci de nettoyer votre feuille, sauvegarder le résultat, et soumettre le *notebook* sur Cyberlearn.